In [1]:
import numpy as np
import pandas as pd

# Load Svensson parameters
df_sv = pd.read_excel("svensson_parameters_cleaned.xlsx", parse_dates=['Date'])
# Ensuring numeric columns are really floats
numeric_cols = ['beta0','beta1','beta2','beta3','tau1','tau2']
df_sv[numeric_cols] = df_sv[numeric_cols].apply(pd.to_numeric, errors='coerce')
# Svensson forward rate function
def forward_rate(m, params):
    β0, β1, β2, β3, τ1, τ2 = params
    term1 = β0
    term2 = β1 * np.exp(-m / τ1)
    term3 = β2 * (m / τ1) * np.exp(-m / τ1)
    term4 = β3 * (m / τ2) * np.exp(-m / τ2)
    return term1 + term2 + term3 + term4

# Spot rate as average of forward rates up to maturity m
def spot_rate(m, params):
    t = np.linspace(0.0001, m, 100)
    fwd = forward_rate(t, params)
    return np.trapz(fwd, t) / m

# Use latest available row
latest = df_sv.iloc[-1]
cols = ['beta0', 'beta1', 'beta2', 'beta3', 'tau1', 'tau2']

# Convert β-values to decimals
params = latest[cols].values
params[:4] = params[:4] / 100  # beta0-3 in percent → decimal

# Compute spot rate for 1 year
r_1y = spot_rate(1.0, params)

# Print result
print(f"1-year spot rate: {r_1y:.4f} ({r_1y:.2%})")

1-year spot rate: 0.0181 (1.81%)


In [2]:
# --- Main input: 6-year spot rate for discounting the certificate ---
# Since the product has a maturity of 6 years, we use the 6-year spot rate from the Svensson model.
# This gives the appropriate risk-free rate to discount expected payoffs over the full product horizon.
r_6y = spot_rate(6.0, params)
print(f"6-year spot rate: {r_6y:.4%}")

6-year spot rate: 2.2752%
